In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=1 , doc_content_chars_max=200)

wikipedia  = WikipediaQueryRun(api_wrapper=wikipedia_wrapper)

In [ ]:
wikipedia.name

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

In [ ]:
docs

In [ ]:
rct = RecursiveCharacterTextSplitter(chunk_size=1000 , chunk_overlap=200)
documents = rct.split_documents(docs)
documents[:20]

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
vectordb = FAISS.from_documents(docs , HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs))

In [ ]:
retriever = vectordb.as_retriever()
retriever

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever , "LangSmith_Search" ,"Search for any queries related to Langchain chains")

In [ ]:
retriever_tool

In [ ]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1 , doc_content_chars_max=200)

arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

In [ ]:
tools = [wikipedia , arxiv , retriever_tool]

In [ ]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_TOGETHER_API')
llm = ChatOpenAI(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
    openai_api_key=OPENAI_API_KEY,
    openai_api_base="https://api.together.xyz/v1",
    temperature = 0
)

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm , tools , prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent , tools=tools , verbose=True)

In [ ]:
agent_executor

In [ ]:
query = agent_executor.invoke({"input":"What is the paper 1605.08386 about"})

In [ ]:
query['output']